# DIOMIRA

## A nb to develop the cython version of DIOMIRA

In [1]:
from __future__ import print_function
%matplotlib inline
%load_ext autoreload
%autoreload 2

In [2]:
%env IPYTHONDIR=/Users/jjgomezcadenas/Documents/Development/IPYTHON

env: IPYTHONDIR=/Users/jjgomezcadenas/Documents/Development/IPYTHON


In [3]:
%load_ext Cython

In [4]:
import pandas as pd
import numpy as np
import FEParam as FP
import tables
from time import time
from Util import *

### Classical fib function

In [5]:
def fib(n):
    a,b = 0.0, 1.0
    for i in range(n):
        a,b = a+b,a
    return a

In [6]:
%%cython
def cfib(int n):
    cdef int i
    cdef double a = 0.0, b= 1.0
    for i in range(n):
        a,b = a+b, a
    return a

In [7]:
%timeit fib(0)

The slowest run took 8.84 times longer than the fastest. This could mean that an intermediate result is being cached.
1000000 loops, best of 3: 351 ns per loop


In [8]:
%timeit fib(90)

100000 loops, best of 3: 6.02 µs per loop


In [9]:
%timeit cfib(0)

The slowest run took 24.58 times longer than the fastest. This could mean that an intermediate result is being cached.
10000000 loops, best of 3: 48.5 ns per loop


In [10]:
%timeit cfib(90)

10000000 loops, best of 3: 119 ns per loop


In [11]:
f = 118./6050.

In [12]:
1./f

51.271186440677965

### Rebin array

#### Python version

In [13]:
def rebin_pmt_array(a,stride):
    """
    rebins pmt array a according to stride
    there is a rebin_array in util which uses
    lenb = (len(a))/int(stride)
    this version uses (lean(a)+1) to correct from the fact that the
    MCRD is 599999 channels (should be 600000)
    """
    
    lenb = (len(a)+1)/int(stride)
    b = np.zeros(lenb)
    j=0
    for i in range(lenb):
        b[i] = np.sum(a[j:j+stride])
        j+= stride
    return b

In [14]:
cx = np.arange(0.,100., 1)

In [15]:
%time y = rebin_pmt_array(cx,10)

CPU times: user 76 µs, sys: 22 µs, total: 98 µs
Wall time: 90.8 µs


In [16]:
%timeit y = rebin_pmt_array(cx,10)

10000 loops, best of 3: 30.9 µs per loop


#### Cython version (pure compilation)

In [17]:
%%cython
import numpy as np
def rebin_pmt_array(a,stride):
    """
    rebins pmt array a according to stride
    there is a rebin_array in util which uses
    lenb = (len(a))/int(stride)
    this version uses (lean(a)+1) to correct from the fact that the
    MCRD is 599999 channels (should be 600000)
    """
    
    lenb = (len(a)+1)/int(stride)
    b = np.zeros(lenb)
    j=0
    for i in range(lenb):
        b[i] = np.sum(a[j:j+stride])
        j+= stride
    return b

In [18]:
%timeit rebin_pmt_array(cx,10)

10000 loops, best of 3: 28.2 µs per loop


#### cython version, declares variables but still uses np.sum()

In [19]:
%%cython
import numpy as np
cimport numpy as np
def crebin_pmt_array(double [:] a,int stride):
    """
    rebins pmt array a according to stride
    there is a rebin_array in util which uses
    lenb = (len(a))/int(stride)
    this version uses (lean(a)+1) to correct from the fact that the
    MCRD is 599999 channels (should be 600000)
    """
    
    cdef:
        int lenb, i, j
        double [:] b
    
    lena =   a.shape[0]   
    lenb = a.shape[0]/stride
    
    b = np.zeros(lenb, dtype=np.double)
    
    j=0
    
    for i in range(lenb):
        b[i] = np.sum(a[j:j+stride])
        j+= stride
    
    
    return np.asarray(b)

In [20]:
%timeit z = crebin_pmt_array(cx,10)

The slowest run took 23.47 times longer than the fastest. This could mean that an intermediate result is being cached.
10000 loops, best of 3: 65.4 µs per loop


*NB: the code runs 3 times SLOWER than the cython version*

#### cythonizes the sum

In [21]:
%%cython
import numpy as np
cimport numpy as np
def c2rebin_pmt_array(double [:] a,int stride):
    """
    rebins pmt array a according to stride
    there is a rebin_array in util which uses
    lenb = (len(a))/int(stride)
    this version uses (lean(a)+1) to correct from the fact that the
    MCRD is 599999 channels (should be 600000)
    """
    
    cdef:
        int lenb, i, j,k
        double [:] b
        double part_sum
    
    lena =   a.shape[0]   
    lenb = a.shape[0]/stride
    
    b = np.zeros(lenb)
    
    j=0
    
    for i in range(lenb):
        part_sum = 0.
        for k in range(j, j + stride): 
            part_sum += a[k] 
        b[i] = part_sum
        j+= stride
    
    return np.asarray(b)

In [22]:
%timeit z = c2rebin_pmt_array(cx,10)

The slowest run took 302.77 times longer than the fastest. This could mean that an intermediate result is being cached.
100000 loops, best of 3: 4.63 µs per loop


*IMPROVEMENT*

#### Direct use of nd.array() in function

In [23]:
%%cython
import numpy as np
cimport numpy as np
def c3rebin_pmt_array(double [:] a,int stride):
    """
    rebins pmt array a according to stride
    there is a rebin_array in util which uses
    lenb = (len(a))/int(stride)
    this version uses (lean(a)+1) to correct from the fact that the
    MCRD is 599999 channels (should be 600000)
    """
    
    cdef:
        int lenb, i, j,k
        double part_sum
      
    lenb = a.shape[0]/stride
    cdef np.ndarray[np.float64_t, ndim=1] b = np.zeros(lenb) 
    
    j=0
    for i in range(lenb):
        part_sum = 0.
        for k in range(j, j + stride): 
            part_sum += a[k] 
        b[i] = part_sum
        j+= stride
    
    return b

In [24]:
%timeit z = c3rebin_pmt_array(cx,10)

The slowest run took 526.03 times longer than the fastest. This could mean that an intermediate result is being cached.
100000 loops, best of 3: 2.66 µs per loop


*Further improvement!*

#### switch off checks

In [25]:
%%cython
import numpy as np
cimport numpy as np
from cython cimport boundscheck, wraparound
def c4rebin_pmt_array(np.ndarray[np.float64_t, ndim=1] a,int stride):
    """
    rebins pmt array a according to stride
    there is a rebin_array in util which uses
    lenb = (len(a))/int(stride)
    this version uses (lean(a)+1) to correct from the fact that the
    MCRD is 599999 channels (should be 600000)
    """
    
    cdef:
        int lenb, i, j,k
        double part_sum
      
    lenb = a.shape[0]/stride
    cdef np.ndarray[np.float64_t, ndim=1] b = np.zeros(lenb) 
    
    j=0
    with boundscheck(False), wraparound(False):
        for i in range(lenb):
            part_sum = 0.
            for k in range(j, j + stride): 
                part_sum += a[k] 
                b[i] = part_sum
            j+= stride
    
    return b

In [26]:
%timeit z = c4rebin_pmt_array(cx,10)

The slowest run took 458.21 times longer than the fastest. This could mean that an intermediate result is being cached.
100000 loops, best of 3: 2.22 µs per loop


In [27]:
%timeit z = c4rebin_pmt_array(cx,10)

The slowest run took 7.85 times longer than the fastest. This could mean that an intermediate result is being cached.
100000 loops, best of 3: 2.19 µs per loop


In [28]:
%timeit z = rebin_pmt_array(cx,10)

10000 loops, best of 3: 29.2 µs per loop


*About a factor 15 improvement*

### Profiling

In [29]:
import cProfile

In [30]:
def main():
    cx = np.arange(0.,1000000., 1)
    z = rebin_pmt_array(cx,10)

In [31]:
cProfile.run('main()', sort='time')

         400005 function calls in 0.356 seconds

   Ordered by: internal time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
   100000    0.162    0.000    0.162    0.000 {method 'reduce' of 'numpy.ufunc' objects}
   100000    0.078    0.000    0.291    0.000 fromnumeric.py:1743(sum)
        1    0.060    0.060    0.352    0.352 {_cython_magic_d5015329a00d8b98de93e435443336df.rebin_pmt_array}
   100000    0.026    0.000    0.189    0.000 _methods.py:31(_sum)
   100000    0.025    0.000    0.025    0.000 {isinstance}
        1    0.004    0.004    0.004    0.004 {numpy.core.multiarray.arange}
        1    0.000    0.000    0.356    0.356 <ipython-input-30-974bca651f3d>:1(main)
        1    0.000    0.000    0.356    0.356 <string>:1(<module>)
        1    0.000    0.000    0.000    0.000 {method 'disable' of '_lsprof.Profiler' objects}




In [32]:
def cmain():
    cx = np.arange(0.,1000000., 1)
    z = c4rebin_pmt_array(cx,10)

In [33]:
cProfile.run('cmain()', sort='time')

         5 function calls in 0.003 seconds

   Ordered by: internal time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.002    0.002    0.002    0.002 {_cython_magic_d19fd7fbfbbb40b8b0630f7e3ab017a6.c4rebin_pmt_array}
        1    0.001    0.001    0.001    0.001 {numpy.core.multiarray.arange}
        1    0.000    0.000    0.003    0.003 <string>:1(<module>)
        1    0.000    0.000    0.003    0.003 <ipython-input-32-7a72f97666b8>:1(cmain)
        1    0.000    0.000    0.000    0.000 {method 'disable' of '_lsprof.Profiler' objects}




*NB It seems that the improvement for large arrays can be much larger***

## DIOMIRA

In [69]:
"""
DIOMIRA
JJGC August 2016

What DIOMIRA does:
1) Reads a MCRD file containing MC waveforms for the 12 PMTs of the EP.
   Each waveform contains number of PEs in bins of 1 ns.
2) Convolves the PE waveform with the response of the FEE electronics.
3) Decimates the waveform, simulating the effect of the DAQ sampling (25 ns bins)
4) Writes a RWF file with the new data and adds the FEE simulation parameters as metadata
"""

from __future__ import print_function
from Util import *
from LogConfig import *
from Configure import configure
from Nh5 import *
from cities import diomira
from SensorsResponse import *

import tables


"""
ChangeLog:

26.9 

Changed types of PMTRWF, SIPMRWF and PMTTWF to Float32 for 
    (compatibility with ART/GATE)

Do not store EPMT and ESIPM (can be computed on the fly)

Change sign of pmtrwf to negative (as produced by the DAQ)
"""

def DIOMIRA(argv):
    DEBUG_LEVEL, INFO, CFP = configure(argv[0],argv[1:])
   
    if INFO:
        print(diomira)

    #wait()
    
    print("""
        DIOMIRA:
         1. Reads an MCRD file produced by art/centella, which stores MCRD 
         waveforms for PMTs (bins of 1 ns)
        and the SiPMs (bins of 1 mus)
            

        2. Simulates the response of the energy plane in the PMTs MCRD, 
        and produces both RWF and TWF:
        see: http://localhost:8931/notebooks/Nh5-Event-Model.ipynb#Reconstructed-Objects
        
            
        3. Simulates the response of the tracking plane in the SiPMs MCRD and outputs
            SiPM RWF (not yet implemented, for the time being simply copy the MCRD)

        4. Add a table describing the FEE parameters used for simulation

        5. Copies the tables on geometry, detector data and MC


        """)
    FP.print_FEE()
    #wait()

    PATH_IN =CFP['PATH_IN']
    PATH_OUT =CFP['PATH_OUT']
    FILE_IN =CFP['FILE_IN']
    FILE_OUT =CFP['FILE_OUT']
    FIRST_EVT =CFP['FIRST_EVT']
    LAST_EVT =CFP['LAST_EVT']
    RUN_ALL =CFP['RUN_ALL']
    CLIB =CFP['CLIB']
    CLEVEL =CFP['CLEVEL']
    NEVENTS = LAST_EVT - FIRST_EVT

    print('Debug level = {}'.format(DEBUG_LEVEL))

    logger.info("input path ={}; output path = {}; file_in ={} file_out ={}".format(
        PATH_IN,PATH_OUT,FILE_IN, FILE_OUT))

    logger.info("first event = {} last event = {} nof events requested = {} ".format(
        FIRST_EVT,LAST_EVT,NEVENTS))

    logger.info("Compression library = {} Compression level = {} ".format(
        CLIB,CLEVEL))

    # open the input file 
    with tables.open_file("{}/{}".format(PATH_IN,FILE_IN), "r+") as h5in: 
        # access the PMT raw data in file 

        pmtrd_ = h5in.root.pmtrd
        sipmrd_ = h5in.root.sipmrd

        #pmtrd_.shape = (nof_events, nof_sensors, wf_length)
        NPMT = pmtrd_.shape[1]
        NSIPM = sipmrd_.shape[1]
        PMTWL = pmtrd_.shape[2] 
        PMTWL_FEE = int((PMTWL+1)/FP.time_DAQ)
        SIPMWL = sipmrd_.shape[2]
        NEVENTS_DST = pmtrd_.shape[0]

        logger.info("nof PMTs = {} nof  SiPMs = {} nof events in input DST = {} ".format(
        NPMT,NSIPM,NEVENTS_DST))

        logger.info("lof SiPM WF = {} lof PMT WF (MC) = {} lof PMT WF (FEE) = {}".format(
        PMTWL,SIPMWL,PMTWL_FEE))

        #wait()

        #access the geometry and the sensors metadata info

        geom_t = h5in.root.Detector.DetectorGeometry
        pmt_t = h5in.root.Sensors.DataPMT
        sipm_t = h5in.root.Sensors.DataSiPM
        mctrk_t = h5in.root.MC.MCTracks

        
        # open the output file 
        with tables.open_file("{}/{}".format(PATH_OUT,FILE_OUT), "w",
            filters=tables.Filters(complib=CLIB, complevel=CLEVEL)) as h5out:
 
            # create a group to store MC data
            mcgroup = h5out.create_group(h5out.root, "MC")
            # copy the mctrk table
            mctrk_t.copy(newparent=mcgroup)

            # create a group  to store geom data
            detgroup = h5out.create_group(h5out.root, "Detector")
            # copy the geom table
            geom_t.copy(newparent=detgroup)

            # create a group  store sensor data
            sgroup = h5out.create_group(h5out.root, "Sensors")
            # copy the pmt table
            pmt_t.copy(newparent=sgroup)
            # copy the sipm table
            sipm_t.copy(newparent=sgroup)

            # create a table to store Energy plane FEE data and hang it from MC group
            fee_table = h5out.create_table(mcgroup, "FEE", FEE,
                          "EP-FEE parameters",
                           tables.Filters(0))

            # fill table
            FEE_param_table(fee_table)

            # create a group to store RawData
            rgroup = h5out.create_group(h5out.root, "RD")
            
            # create an extensible array to store the RWF waveforms
            pmtrwf = h5out.create_earray(h5out.root.RD, "pmtrwf", 
                                    atom=tables.Float32Atom(), 
                                    shape=(0, NPMT, PMTWL_FEE), 
                                    expectedrows=NEVENTS_DST)
            
            # create an extensible array to store the TWF waveforms
            pmttwf = h5out.create_earray(h5out.root.RD, "pmttwf", 
                                    atom=tables.Float32Atom(), 
                                    shape=(0, NPMT, PMTWL_FEE), 
                                    expectedrows=NEVENTS_DST)
            

            sipmrwf = h5out.create_earray(h5out.root.RD, "sipmrwf", 
                                    atom=tables.Float32Atom(), 
                                    shape=(0, NSIPM, SIPMWL), 
                                    expectedrows=NEVENTS_DST)

            # #create an extensible array to store the energy in PES of PMTs 
            # epmt = h5out.create_earray(h5out.root.RD, "epmt", 
            #                         atom=tables.FloatAtom(), 
            #                         shape=(0, NPMT), 
            #                         expectedrows=NEVENTS_DST)

            # # create an extensible array to store the energy in PES of SiPMs 
            # esipm = h5out.create_earray(h5out.root.RD, "esipm", 
            #                         atom=tables.FloatAtom(), 
            #                         shape=(0, NSIPM), 
            #                         expectedrows=NEVENTS_DST)

            
            if NEVENTS > NEVENTS_DST and RUN_ALL == False:
                print("""
                Refusing to run: you have requested
                FIRST_EVT = {}
                LAST_EVT  = {}
                Thus you want to run over {} events
                but the size of the DST is {} events.
                Please change your choice or select RUN_ALL = TRUE
                to run over the whole DST when this happens
                """.format(FIRST_EVT,LAST_EVT,NEVENTS,NEVENTS_DST))
                sys.exit(0)
            elif  NEVENTS > NEVENTS_DST and RUN_ALL == True:
                FIRST_EVT = 0
                LAST_EVT = NEVENTS_DST
                NEVENTS = NEVENTS_DST


            for i in range(FIRST_EVT,LAST_EVT):
                logger.info("-->event number ={}".format(i))

                #simulate PMT response and return an array with RWF
                dataPMT = simulate_pmt_response(i,pmtrd_)

                #convert to float
                dataPMT.astype(float) 
                #TWF
                 
                truePMT = rebin_signal(i,pmtrd_, int(FP.time_DAQ))
                truePMT.astype(float)
                
                logger.info("truePMT shape ={}".format(truePMT.shape))
                logger.info("dataPMT shape ={}".format(dataPMT.shape))
                
                #RWF for pmts
                pmtrwf.append(dataPMT.reshape(1, NPMT, PMTWL_FEE))
                #pmtrd.append(dataPMT.reshape(1, NPMT, PMTWL))
                
                #TWF for pmts
                pmttwf.append(truePMT.reshape(1, NPMT, PMTWL_FEE))
                #pmtrd.append(dataPMT.reshape(1, NPMT, PMTWL))
                   
                #simulate SiPM response and return an array with new WF
                dataSiPM = simulate_sipm_response(i,sipmrd_)
                dataSiPM.astype(float)
                
                #append to SiPM EARRAY
                sipmrwf.append(dataSiPM.reshape(1, NSIPM, SIPMWL))

                # #fill ene_pmt vector
                # enePMT = energy_pes(i, pmtrd_)
                # #append to epmt EARRAY
                # epmt.append(enePMT.reshape(1, NPMT))

                # #fill ene_sipm vector
                # eneSIPM = energy_pes(i, sipmrd_)
                # esipm.append(eneSIPM.reshape(1, NSIPM))

            pmtrwf.flush()
            pmttwf.flush()
            sipmrwf.flush()
            #epmt.flush()
            #esipm.flush()


    print("Leaving Diomira. Safe travels!")



  

In [35]:
%less ../../Config/DIOMIRA_NA_ZLIB_test2.csv

### Profile DIOMIRA

In [70]:
cProfile.run("DIOMIRA(['DIOMIRA','-i','-d','INFO','-c','../../Config/DIOMIRA_NA_ZLIB_test2.csv'])", sort='time')

INFO:root:Configuration Parameters (CFP) dictionary  = {'FIRST_EVT': 0, 'LAST_EVT': 3, 'FILE_OUT': 'WF_Na_ZLIB_test3_RWF.h5', 'PATH_OUT': '/Users/jjgomezcadenas/Documents/Development/NEXT/data/Waveforms/25ns/', ' END ': 1, 'CLIB': 'zlib', 'RUN_ALL': 1, 'PATH_IN': '/Users/jjgomezcadenas/Documents/Development/NEXT/data/Waveforms/WF-NA-ZLIB/', 'CLEVEL': 1, 'FILE_IN': 'WF_Na_1Kevts_comp1_chunk32k.h5'}


Configuration Parameters (CFP) dictionary  = {'FIRST_EVT': 0, 'LAST_EVT': 3, 'FILE_OUT': 'WF_Na_ZLIB_test3_RWF.h5', 'PATH_OUT': '/Users/jjgomezcadenas/Documents/Development/NEXT/data/Waveforms/25ns/', ' END ': 1, 'CLIB': 'zlib', 'RUN_ALL': 1, 'PATH_IN': '/Users/jjgomezcadenas/Documents/Development/NEXT/data/Waveforms/WF-NA-ZLIB/', 'CLEVEL': 1, 'FILE_IN': 'WF_Na_1Kevts_comp1_chunk32k.h5'}



Leaving there and proceeding for three days toward the east, you reach Diomira, 
a city with sixty silver domes, bronze statues of all the gods, streets paved with lead, 
a crystal theater, a golden cock that crows every morning on a tower. 
All these beauties will already be familiar to the visitor, 
who has seen them also in other cities. 
But the special quality of this city for the man who arrives there on a September evening, 
when the days are growing shorter 
and the multicolored lamps are lighted all at once at the doors of the food stalls 
and from a terrace a woman's voice cries ooh!, is that he feels envy 
toward those who now believe they have once before lived an 
evening identical to this and who think they were happy, that time.

        DIOMIRA:
         1. Reads an MCRD file produced by art/centella, which stores MCRD 
         waveforms for PMTs (bins of 1 ns)
        and the SiPMs (bins of 1 mus)
            

        2. Simulates the response of the energy plane in

input path =/Users/jjgomezcadenas/Documents/Development/NEXT/data/Waveforms/WF-NA-ZLIB/; output path = /Users/jjgomezcadenas/Documents/Development/NEXT/data/Waveforms/25ns/; file_in =WF_Na_1Kevts_comp1_chunk32k.h5 file_out =WF_Na_ZLIB_test3_RWF.h5


INFO:root:first event = 0 last event = 3 nof events requested = 3 


...

INFO:root:truePMT shape =(12, 24000)


truePMT shape =(12, 24000)


INFO:root:dataPMT shape =(12, 24000)


dataPMT shape =(12, 24000)


Leaving Diomira. Safe travels!
         3699711 function calls (3699682 primitive calls) in 6.610 seconds

   Ordered by: internal time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
   865310    1.584    0.000    1.584    0.000 {method 'reduce' of 'numpy.ufunc' objects}
       36    0.689    0.019    0.689    0.019 {method 'normal' of 'mtrand.RandomState' objects}
      468    0.681    0.001    0.681    0.001 {numpy.core.multiarray.correlate}
   864039    0.674    0.000    2.693    0.000 fromnumeric.py:1743(sum)
       36    0.653    0.018    3.364    0.093 SensorsResponse.py:127(rebin_pmt_array)
        7    0.290    0.041    0.290    0.041 {method '_fill_col' of 'tables.tableextension.Row' objects}
       72    0.258    0.004    0.258    0.004 {scipy.signal.sigtools._linear_filter}
        8    0.250    0.031    0.250    0.031 {method '_append_records' of 'tables.tableextension.Table' objects}
     5448    0.240    0.000    0.240    0.000 {method '_g_read_s

*Profiling shows that rebin_array is one of the main issues. Start cythonizing it

In [95]:
def FEE_param_table(fee_table):
    """
    Stores the parameters of the EP FEE simulation 
    """
    
    print("calling the cython version")
    
    row = fee_table.row
    row['offset'] = FP.offset
    row['pmt_gain'] = FP.PMT_GAIN
    row['V_gain'] = FP.V_GAIN
    row['R'] = FP.R
    row['C12'] = FP.C12
    row['CO12'] = FP.C12 # to be rewritten by ISIDORA
    row['time_step'] = FP.time_step
    row['time_daq'] = FP.time_DAQ
    row['freq_LPF'] = FP.freq_LPF
    row['freq_HPF'] = 1./(2*pi*FP.R*FP.C)
    row['LSB'] = FP.LSB
    row['volts_to_adc'] = FP.voltsToAdc/volt
    row['noise_fee_rms'] = FP.NOISE_FEE
    row['noise_adc'] = FP.NOISE_ADC
    
    row.append()
def DIOMIRA_cy(argv):
    DEBUG_LEVEL, INFO, CFP = configure(argv[0],argv[1:])
   
    if INFO:
        print(diomira)

    #wait()
    
    print("""
        DIOMIRA:
         1. Reads an MCRD file produced by art/centella, which stores MCRD 
         waveforms for PMTs (bins of 1 ns)
        and the SiPMs (bins of 1 mus)
            

        2. Simulates the response of the energy plane in the PMTs MCRD, 
        and produces both RWF and TWF:
        see: http://localhost:8931/notebooks/Nh5-Event-Model.ipynb#Reconstructed-Objects
        
            
        3. Simulates the response of the tracking plane in the SiPMs MCRD and outputs
            SiPM RWF (not yet implemented, for the time being simply copy the MCRD)

        4. Add a table describing the FEE parameters used for simulation

        5. Copies the tables on geometry, detector data and MC


        """)
    FP.print_FEE()
    #wait()

    PATH_IN =CFP['PATH_IN']
    PATH_OUT =CFP['PATH_OUT']
    FILE_IN =CFP['FILE_IN']
    FILE_OUT =CFP['FILE_OUT']
    FIRST_EVT =CFP['FIRST_EVT']
    LAST_EVT =CFP['LAST_EVT']
    RUN_ALL =CFP['RUN_ALL']
    CLIB =CFP['CLIB']
    CLEVEL =CFP['CLEVEL']
    NEVENTS = LAST_EVT - FIRST_EVT

    print('Debug level = {}'.format(DEBUG_LEVEL))

    logger.info("input path ={}; output path = {}; file_in ={} file_out ={}".format(
        PATH_IN,PATH_OUT,FILE_IN, FILE_OUT))

    logger.info("first event = {} last event = {} nof events requested = {} ".format(
        FIRST_EVT,LAST_EVT,NEVENTS))

    logger.info("Compression library = {} Compression level = {} ".format(
        CLIB,CLEVEL))

    # open the input file 
    with tables.open_file("{}/{}".format(PATH_IN,FILE_IN), "r+") as h5in: 
        # access the PMT raw data in file 

        pmtrd_ = h5in.root.pmtrd
        sipmrd_ = h5in.root.sipmrd

        #pmtrd_.shape = (nof_events, nof_sensors, wf_length)
        NPMT = pmtrd_.shape[1]
        NSIPM = sipmrd_.shape[1]
        PMTWL = pmtrd_.shape[2] 
        PMTWL_FEE = int((PMTWL+1)/FP.time_DAQ)
        SIPMWL = sipmrd_.shape[2]
        NEVENTS_DST = pmtrd_.shape[0]

        logger.info("nof PMTs = {} nof  SiPMs = {} nof events in input DST = {} ".format(
        NPMT,NSIPM,NEVENTS_DST))

        logger.info("lof SiPM WF = {} lof PMT WF (MC) = {} lof PMT WF (FEE) = {}".format(
        PMTWL,SIPMWL,PMTWL_FEE))

        #wait()

        #access the geometry and the sensors metadata info

        geom_t = h5in.root.Detector.DetectorGeometry
        pmt_t = h5in.root.Sensors.DataPMT
        sipm_t = h5in.root.Sensors.DataSiPM
        mctrk_t = h5in.root.MC.MCTracks

        
        # open the output file 
        with tables.open_file("{}/{}".format(PATH_OUT,FILE_OUT), "w",
            filters=tables.Filters(complib=CLIB, complevel=CLEVEL)) as h5out:
 
            # create a group to store MC data
            mcgroup = h5out.create_group(h5out.root, "MC")
            # copy the mctrk table
            mctrk_t.copy(newparent=mcgroup)

            # create a group  to store geom data
            detgroup = h5out.create_group(h5out.root, "Detector")
            # copy the geom table
            geom_t.copy(newparent=detgroup)

            # create a group  store sensor data
            sgroup = h5out.create_group(h5out.root, "Sensors")
            # copy the pmt table
            pmt_t.copy(newparent=sgroup)
            # copy the sipm table
            sipm_t.copy(newparent=sgroup)

            # create a table to store Energy plane FEE data and hang it from MC group
            fee_table = h5out.create_table(mcgroup, "FEE", FEE,
                          "EP-FEE parameters",
                           tables.Filters(0))

            # fill table
            FEE_param_table(fee_table)

            # create a group to store RawData
            rgroup = h5out.create_group(h5out.root, "RD")
            
            # create an extensible array to store the RWF waveforms
            pmtrwf = h5out.create_earray(h5out.root.RD, "pmtrwf", 
                                    atom=tables.Float32Atom(), 
                                    shape=(0, NPMT, PMTWL_FEE), 
                                    expectedrows=NEVENTS_DST)
            
            # create an extensible array to store the TWF waveforms
            pmttwf = h5out.create_earray(h5out.root.RD, "pmttwf", 
                                    atom=tables.Float32Atom(), 
                                    shape=(0, NPMT, PMTWL_FEE), 
                                    expectedrows=NEVENTS_DST)
            

            sipmrwf = h5out.create_earray(h5out.root.RD, "sipmrwf", 
                                    atom=tables.Float32Atom(), 
                                    shape=(0, NSIPM, SIPMWL), 
                                    expectedrows=NEVENTS_DST)

            
            if NEVENTS > NEVENTS_DST and RUN_ALL == False:
                print("""
                Refusing to run: you have requested
                FIRST_EVT = {}
                LAST_EVT  = {}
                Thus you want to run over {} events
                but the size of the DST is {} events.
                Please change your choice or select RUN_ALL = TRUE
                to run over the whole DST when this happens
                """.format(FIRST_EVT,LAST_EVT,NEVENTS,NEVENTS_DST))
                sys.exit(0)
            elif  NEVENTS > NEVENTS_DST and RUN_ALL == True:
                FIRST_EVT = 0
                LAST_EVT = NEVENTS_DST
                NEVENTS = NEVENTS_DST


            for i in range(FIRST_EVT,LAST_EVT):
                logger.info("-->event number ={}".format(i))
                
                
                logger.info("-->calling simulate_pmt_response_cy")
                #simulate PMT response and return an array with RWF
                dataPMT = simulate_pmt_response_cy(i,pmtrd_)

                #convert to float
                dataPMT.astype(float) 
                #TWF
                logger.info("-->calling rebin_signal_cy")
                truePMT = rebin_signal_cy(i,pmtrd_, int(FP.time_DAQ))
                truePMT.astype(float)
                
                logger.info("truePMT shape ={}".format(truePMT.shape))
                logger.info("dataPMT shape ={}".format(dataPMT.shape))
                
                #RWF for pmts
                pmtrwf.append(dataPMT.reshape(1, NPMT, PMTWL_FEE))
                
                
                #TWF for pmts
                pmttwf.append(truePMT.reshape(1, NPMT, PMTWL_FEE))
                
                   
                #simulate SiPM response and return an array with new WF
                dataSiPM = simulate_sipm_response_cy(i,sipmrd_)
                dataSiPM.astype(float)
                
                #append to SiPM EARRAY
                sipmrwf.append(dataSiPM.reshape(1, NSIPM, SIPMWL))


            pmtrwf.flush()
            pmttwf.flush()
            sipmrwf.flush()


    print("Leaving Diomira. Safe travels!")







In [85]:
DIOMIRA_cy

<function __main__.DIOMIRA_cy>

In [99]:
%%cython

from Util import *
from LogConfig import *
import FEParam as FP
import SPE as SP
import FEE2 as FE

import numpy as np
cimport numpy as np
from cython cimport boundscheck, wraparound

cpdef np.ndarray rebin_array_cy3(np.ndarray[int,ndim=1] a, int stride):
    """
    rebins array a according to stride
    """
    
    logger.info("-->rebin_array_cy3")
    cdef int lenb = (len(a)+1)/int(stride)
    b = np.zeros(lenb, dtype=np.int)
    #cdef np.ndarray[int, ndim=1] b = np.empty(lenb, dtype=np.int32)
    cdef int j=0
    cdef int i,k
    cdef int part_sum
    with boundscheck(False), wraparound(False):
        for i in range(lenb):
            part_sum = 0
            for k in range(j, j + stride): 
                part_sum += a[k] 
                b[i] = part_sum
            j+= stride
            
    return b

cpdef np.ndarray rebin_signal_cy3(int event_number,pmtrd_):
    """
    rebins the MCRD signal to produce TWF (pes, bins 25 ns)
    """

    cdef list rdata = []
    
    #cdef np.ndarray[int,ndim=1] pmt
    #cdef np.ndarray[int,ndim=1] twf
    cdef int j
    
    logger.info("-->rebin_signal_cy3")
    for j in range(pmtrd_.shape[1]):
        logger.info("-->PMT number ={}".format(j))
                
        pmt = pmtrd_[event_number, j] #waveform for event event_number, PMT j
        twf = rebin_array_cy3(pmt, int(FP.time_DAQ))
        
        rdata.append(twf)
    return np.array(rdata)



def simulate_sipm_response_cy(event_number,sipmrd_):
    """
    For the moment use a dummy rutne that simply copies the sipm EARRAY
    """
    rdata = []

    for j in range(sipmrd_.shape[1]):
        rdata.append(sipmrd_[event_number, j])
    return np.array(rdata)


def simulate_pmt_response_cy(event_number,pmtrd_):
    """
    Sensor Response
    Given a signal in PE (photoelectrons in bins of 1 ns) and the response function of 
    for a single photoelectron (spe) and front-end electronics (fee)
    this function produces the PMT raw data (adc counts bins 25 ns)

    pmtrd_ dataset that holds the PMT PE data for each PMT
    pmtrd25 dataset to be created with adc counts, bins 25 ns 
    after convoluting with electronics
    """
    
    logger.info("-->*cython::simulate_pmt_response, event ={}".format(event_number))
  
    rdata = []

    for j in range(pmtrd_.shape[1]):
        logger.info("-->PMT number ={}".format(j))
                
        pmt = pmtrd_[event_number, j] #waveform for event event_number, PMT j
        
        fee = FE.FEE(C=FP.C12[j],R= FP.R, f=FP.freq_LPF, RG=FP.V_GAIN) 
        spe = SP.SPE(pmt_gain=FP.PMT_GAIN,x_slope = 5*ns,x_flat = 1*ns)
    
        signal_PMT = spe.SpePulseFromVectorPE(pmt) #PMT response

        #Front end response to PMT pulse (in volts)
        signal_fee = fee.FEESignal(signal_PMT, noise_rms=FP.NOISE_FEE) 

        #Signal out of DAQ
        #positive signal convention
        #signal_daq = fee.daqSignal(signal_fee, noise_rms=0) - FP.offset
        #negative signals convention!

        signal_daq = FP.offset -fee.daqSignal(signal_fee, noise_rms=0) 
    
        rdata.append(signal_daq)
    return np.array(rdata)

def rebin_pmt_array_cy(a,stride):
    """
    rebins pmt array a according to stride
    there is a rebin_array in util which uses
    lenb = (len(a))/int(stride)
    this version uses (lean(a)+1) to correct from the fact that the
    MCRD is 599999 channels (should be 600000)
    """
    
    lenb = (len(a)+1)/int(stride)
    b = np.zeros(lenb)
    j=0
    for i in range(lenb):
        b[i] = np.sum(a[j:j+stride])
        j+= stride
    return b

cdef rebin_pmt_array_cy2(np.ndarray[np.int32_t, ndim=1] a,int stride):
    """
    rebins pmt array a according to stride
    there is a rebin_array in util which uses
    lenb = (len(a))/int(stride)
    this version uses (lean(a)+1) to correct from the fact that the
    MCRD is 599999 channels (should be 600000)
    """
    
    cdef int lenb, i, j,k
    cdef int part_sum
    
    #logger.info("-->*rebin_pmt_array_cy2: after cdef*")
    
      
    lenb = (a.shape[0]+1)/stride
    b = np.zeros(lenb, dtype=np.int32)
    
    #logger.info("--> lenb = {}".format(lenb))
    
    #cdef int[:] b = np.zeros(lenb) 
    #cdef np.ndarray[np.int32_t, ndim=1] b = np.zeros(lenb) 
    
    #logger.info("-->b = {}".format(b[1000:1100]))
    
    #logger.info("-->*rebin_pmt_array_cy2: before loop*")
    j=0
    with boundscheck(False), wraparound(False):
        for i in range(lenb):
            part_sum = 0
            for k in range(j, j + stride): 
                part_sum += a[k] 
                b[i] = part_sum
            j+= stride
    
    return b

def rebin_signal_cy(event_number,pmtrd_, stride):
    """
    rebins the MCRD signal to produce TWF (pes, bins 25 ns)
    """
    
    cdef list rdata = []
    cdef int j
    logger.info("-->*cython::rebin_signal, event ={}".format(event_number))

    for j in range(pmtrd_.shape[1]):
        logger.info("-->PMT number ={}".format(j))
                
        pmt = pmtrd_[event_number, j] #waveform for event event_number, PMT j
        twf = rebin_pmt_array_cy2(pmt, stride)
        
        rdata.append(twf)
    return np.array(rdata)

  


In [87]:
DIOMIRA_cy

<function __main__.DIOMIRA_cy>

In [100]:
cProfile.run("DIOMIRA_cy(['DIOMIRA','-i','-d','INFO','-c','../../Config/DIOMIRA_NA_ZLIB_test2.csv'])", sort='time')

INFO:root:Configuration Parameters (CFP) dictionary  = {'FIRST_EVT': 0, 'LAST_EVT': 3, 'FILE_OUT': 'WF_Na_ZLIB_test3_RWF.h5', 'PATH_OUT': '/Users/jjgomezcadenas/Documents/Development/NEXT/data/Waveforms/25ns/', ' END ': 1, 'CLIB': 'zlib', 'RUN_ALL': 1, 'PATH_IN': '/Users/jjgomezcadenas/Documents/Development/NEXT/data/Waveforms/WF-NA-ZLIB/', 'CLEVEL': 1, 'FILE_IN': 'WF_Na_1Kevts_comp1_chunk32k.h5'}


Configuration Parameters (CFP) dictionary  = {'FIRST_EVT': 0, 'LAST_EVT': 3, 'FILE_OUT': 'WF_Na_ZLIB_test3_RWF.h5', 'PATH_OUT': '/Users/jjgomezcadenas/Documents/Development/NEXT/data/Waveforms/25ns/', ' END ': 1, 'CLIB': 'zlib', 'RUN_ALL': 1, 'PATH_IN': '/Users/jjgomezcadenas/Documents/Development/NEXT/data/Waveforms/WF-NA-ZLIB/', 'CLEVEL': 1, 'FILE_IN': 'WF_Na_1Kevts_comp1_chunk32k.h5'}



Leaving there and proceeding for three days toward the east, you reach Diomira, 
a city with sixty silver domes, bronze statues of all the gods, streets paved with lead, 
a crystal theater, a golden cock that crows every morning on a tower. 
All these beauties will already be familiar to the visitor, 
who has seen them also in other cities. 
But the special quality of this city for the man who arrives there on a September evening, 
when the days are growing shorter 
and the multicolored lamps are lighted all at once at the doors of the food stalls 
and from a terrace a woman's voice cries ooh!, is that he feels envy 
toward those who now believe they have once before lived an 
evening identical to this and who think they were happy, that time.

        DIOMIRA:
         1. Reads an MCRD file produced by art/centella, which stores MCRD 
         waveforms for PMTs (bins of 1 ns)
        and the SiPMs (bins of 1 mus)
            

        2. Simulates the response of the energy plane in

input path =/Users/jjgomezcadenas/Documents/Development/NEXT/data/Waveforms/WF-NA-ZLIB/; output path = /Users/jjgomezcadenas/Documents/Development/NEXT/data/Waveforms/25ns/; file_in =WF_Na_1Kevts_comp1_chunk32k.h5 file_out =WF_Na_ZLIB_test3_RWF.h5


INFO:root:first event = 0 last event = 3 nof events requested = 3 


...

INFO:root:truePMT shape =(12, 24000)


truePMT shape =(12, 24000)


INFO:root:dataPMT shape =(12, 24000)


dataPMT shape =(12, 24000)


Leaving Diomira. Safe travels!
         285044 function calls (285015 primitive calls) in 4.430 seconds

   Ordered by: internal time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        3    1.046    0.349    1.182    0.394 {_cython_magic_fda0d90f18fae9962a2b4688951b7ca3.rebin_signal_cy}
      468    0.700    0.001    0.700    0.001 {numpy.core.multiarray.correlate}
       36    0.695    0.019    0.695    0.019 {method 'normal' of 'mtrand.RandomState' objects}
        7    0.313    0.045    0.313    0.045 {method '_fill_col' of 'tables.tableextension.Row' objects}
        8    0.276    0.034    0.276    0.034 {method '_append_records' of 'tables.tableextension.Table' objects}
       72    0.259    0.004    0.259    0.004 {scipy.signal.sigtools._linear_filter}
     5448    0.245    0.000    0.245    0.000 {method '_g_read_slice' of 'tables.hdf5extension.Array' objects}
        3    0.107    0.036    2.193    0.731 {_cython_magic_fda0d90f18fae9962a2b4688951b7

In [ ]:
%%cython
import numpy as np
cimport numpy as np
from cython cimport boundscheck, wraparound
def rebin_pmt_array(np.ndarray[np.float64_t, ndim=1] a,int stride):
    """
    rebins pmt array a according to stride
    there is a rebin_array in util which uses
    lenb = (len(a))/int(stride)
    this version uses (lean(a)+1) to correct from the fact that the
    MCRD is 599999 channels (should be 600000)
    """
    
    print("*cyhon version*")
    cdef:
        int lenb, i, j,k
        double part_sum
      
    lenb = (a.shape[0]+1)/stride
    cdef np.ndarray[np.float64_t, ndim=1] b = np.zeros(lenb) 
    
    j=0
    with boundscheck(False), wraparound(False):
        for i in range(lenb):
            part_sum = 0.
            for k in range(j, j + stride): 
                part_sum += a[k] 
                b[i] = part_sum
            j+= stride
    
    return b

In [ ]:
rebin_pmt_array

In [ ]:
cProfile.run("DIOMIRA(['DIOMIRA','-i','-d','INFO','-c','../../Config/DIOMIRA_NA_ZLIB_test2.csv'])", sort='time')

In [ ]:
h5f =tables.open_file('/Users/jjgomezcadenas/Documents/Development/NEXT/data/Waveforms/25ns/WF_Na_ZLIB_test100_RWF.h5')

In [ ]:
pmtrwf = h5f.root.RD.pmtrwf

### Python version

In [ ]:

def rebin_pmt_array(a,stride):
    """
    rebins pmt array a according to stride
    there is a rebin_array in util which uses
    lenb = (len(a))/int(stride)
    this version uses (lean(a)+1) to correct from the fact that the
    MCRD is 599999 channels (should be 600000)
    """
    #t1 = time()
    #print("rebin_pmt_array")
    lenb = (len(a)+1)/int(stride)
    b = np.zeros(lenb)
    j=0
    for i in range(lenb):
        b[i] = np.sum(a[j:j+stride])
        j+= stride
    #t2 = time()
    #print('time = {}'.format(t2-t1))
    return b

def rebin_signal(event_number,pmtrd_, stride):
    """
    rebins the MCRD signal to produce TWF (pes, bins 25 ns)
    """
    t1 = time()
    rdata = []

    for j in range(pmtrd_.shape[1]):
        pmt = pmtrd_[event_number, j] #waveform for event event_number, PMT j
        #print("pmt = {}: Call rebin_pmt_array".format(j))
        t2 = time()
        twf = rebin_pmt_array(pmt, stride)
        #print('time = {}'.format(t2-t1))
        
        rdata.append(twf)
    return np.array(rdata)

def rebin_signals(pmtrd, stride=40, list_of_events=[0]):
    """
    rebin the signals in a list of events
    """
    t1 = time()
    for event in list_of_events:
        print("event = {}. Call rebin signal".format(event))
        rebin_signal(event,pmtrd, stride)
        t2 = time()
        print('time = {}'.format(t2-t1))
        
        

In [ ]:
rebin_signals

In [ ]:
%time rebin_signals(pmtrwf, stride=40, list_of_events=range(100))

### Cython version (just compile python)

In [ ]:
%%cython
import pandas as pd
import numpy as np
import FEParam as FP
import tables

def rebin_pmt_array(a,stride):
    """
    rebins pmt array a according to stride
    there is a rebin_array in util which uses
    lenb = (len(a))/int(stride)
    this version uses (lean(a)+1) to correct from the fact that the
    MCRD is 599999 channels (should be 600000)
    """
    
    lenb = (len(a)+1)/int(stride)
    b = np.zeros(lenb)
    j=0
    for i in range(lenb):
        b[i] = np.sum(a[j:j+stride])
        j+= stride
    return b

def rebin_signal(event_number,pmtrd_, stride):
    """
    rebins the MCRD signal to produce TWF (pes, bins 25 ns)
    """
    
    rdata = []

    for j in range(pmtrd_.shape[1]):
        pmt = pmtrd_[event_number, j] #waveform for event event_number, PMT j
        twf = rebin_pmt_array(pmt, stride)
        
        rdata.append(twf)
    return np.array(rdata)

def rebin_signals(pmtrd, stride=40, list_of_events=[0]):
    """
    rebin the signals in a list of events
    """
    for event in list_of_events:
        rebin_signal(event,pmtrd, stride)

In [ ]:
rebin_signals

In [ ]:
def rebin_signal(event_number,pmtrd_, stride):
    """
    rebins the MCRD signal to produce TWF (pes, bins 25 ns)
    """
    t1 = time()
    rdata = []

    for j in range(pmtrd_.shape[1]):
        pmt = pmtrd_[event_number, j] #waveform for event event_number, PMT j
        #print("pmt = {}: Call rebin_pmt_array".format(j))
        t2 = time()
        twf = rebin_pmt_array(pmt, stride)
        #print('time = {}'.format(t2-t1))
        
        rdata.append(twf)
    return np.array(rdata)

def rebin_signals(pmtrd, stride=40, list_of_events=[0]):
    """
    rebin the signals in a list of events
    """
    t1 = time()
    for event in list_of_events:
        print("event = {}. Call rebin signal".format(event))
        rebin_signal(event,pmtrd, stride)
        t2 = time()
        print('time = {}'.format(t2-t1))
        
        

In [ ]:
rebin_signals

In [ ]:
rebin_signal

In [ ]:
rebin_pmt_array

In [ ]:
3000.*25./1000.

In [ ]:
%time rebin_signals(pmtrwf, stride=40, list_of_events=range(100))

### Cython version (supposedly optimized)

In [ ]:
%%cython
import pandas as pd
import numpy as np
import FEParam as FP
import tables
from time import time

def rebin_pmt_array(float [:] a,int stride):
    """
    rebins pmt array a according to stride
    there is a rebin_array in util which uses
    lenb = (len(a))/int(stride)
    this version uses (lean(a)+1) to correct from the fact that the
    MCRD is 599999 channels (should be 600000)
    """
    cdef int lenb
    lenb = (len(a)+1)/stride
    b = np.zeros(lenb, dtype = np.float)
    cdef int j=0
    cdef int i
    for i in range(lenb):
        b[i] = np.sum(a[j:j+stride])
        j+= stride
    return b

# def rebin_signal(int event_number, pmtrd_, int stride):
#     """
#     rebins the MCRD signal to produce TWF (pes, bins 25 ns)
#     """
    
#     #rdata = np.zeros(pmtrd_.shape[1], dtype = np.object)
#     rdata = []
#     cdef int j
#     for j in range(pmtrd_.shape[1]):
#         pmt = pmtrd_[event_number, j] #waveform for event event_number, PMT j
#         twf = rebin_pmt_array(pmt, stride)
        
#         rdata.append(twf)
#     return np.array(rdata)

# def rebin_signals(pmtrd, int stride, list_of_events):
#     """
#     rebin the signals in a list of events
#     """
#     cdef int event
#     for event in list_of_events:
#         rebin_signal(event,pmtrd, stride)

In [ ]:
rebin_pmt_array

In [ ]:
%timeit rebin_signals(pmtrwf, stride=40, list_of_events=range(100))

### Cython does not improve over python code in this case!

In [ ]:
h5f.close()